In [313]:
from tensorflow import keras
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from keras.layers import Normalization
from keras.models import Sequential
from keras.layers import Dense, SimpleRNN, Flatten
from keras import layers
pd.set_option('display.max_rows', 1000)


In [314]:
df_merged.index

MultiIndex([('2021-04-26',              'AFD'),
            ('2021-04-26',              'CDU'),
            ('2021-04-26',              'FDP'),
            ('2021-04-26',           'GRUENE'),
            ('2021-04-26',            'LINKE'),
            ('2021-04-26',              'SPD'),
            ('2021-04-26', 'Surveyed_Persons'),
            ('2021-04-26',            'other'),
            ('2021-04-27',              'AFD'),
            ('2021-04-27',              'CDU'),
            ...
            ('2021-08-23', 'Surveyed_Persons'),
            ('2021-08-23',            'other'),
            ('2021-08-24',              'AFD'),
            ('2021-08-24',              'CDU'),
            ('2021-08-24',              'FDP'),
            ('2021-08-24',           'GRUENE'),
            ('2021-08-24',            'LINKE'),
            ('2021-08-24',              'SPD'),
            ('2021-08-24', 'Surveyed_Persons'),
            ('2021-08-24',            'other')],
           names=['crea

In [328]:
df_merged = pd.read_csv("final_2021_v3.csv")

In [329]:
df_merged["created_at"] = pd.to_datetime(df_merged["created_at"])

In [330]:
df_merged = df_merged.groupby([pd.Grouper(key='created_at',freq='D'), "party"]).agg({
    "replies_count": "sum", 
    "retweets_count": "sum",
    "likes_count": "sum",
    "avg_len_of_tweet": "mean" ,
    "share_of_tweets" :"mean",
    "share_unique_users": "mean",
    "weighted_sentiment" : "mean",
    "share_of_positive_tweets": "mean",
    "share_of_negative_tweets" : "mean",
    "Polls": "mean"}) 

In [331]:
df_merged = df_merged.replace(np.nan, -9999)

In [359]:
model = Sequential()
model.add(layers.Masking(mask_value=-9999., input_shape=(7,10)))
model.add(layers.LSTM(units=20, activation='tanh', return_sequences=True))
model.add(layers.LSTM(units=20, activation='tanh', return_sequences=False))
model.add(layers.Dense(50, activation='relu'))
model.add(layers.Dropout(0.3))
model.add(layers.Dense(7, activation='linear'))


In [381]:
model.summary()

Model: "sequential_23"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
masking_16 (Masking)         (None, 7, 10)             0         
_________________________________________________________________
lstm_39 (LSTM)               (None, 7, 20)             2480      
_________________________________________________________________
lstm_40 (LSTM)               (None, 20)                3280      
_________________________________________________________________
dense_31 (Dense)             (None, 50)                1050      
_________________________________________________________________
dropout_16 (Dropout)         (None, 50)                0         
_________________________________________________________________
dense_32 (Dense)             (None, 7)                 357       
Total params: 7,167
Trainable params: 7,167
Non-trainable params: 0
___________________________________________________

In [360]:
df_merged_test = df_merged.iloc[616:623]

In [361]:
df_merged_test_numpy = df_merged_test.to_numpy().reshape(1,7,10)

In [362]:
df_merged.shape

(847, 10)

In [363]:
df_merged_numpy = df_merged.to_numpy().reshape(121,7,10)

In [364]:
y = df_merged["Polls"]

In [365]:
y_numpy = y.to_numpy().reshape(121,7)

In [376]:


model.compile(loss='mse', optimizer='adam', metrics=['mae'])

model.fit(df_merged_numpy, y_numpy ,
              epochs=25, 
              batch_size=1, 
              verbose=1, 
              validation_split=0.2)



Epoch 1/25
96/96 [==============================] - 10s 29ms/step - loss: 38.3990 - mae: 4.2157 - val_loss: 78.1912 - val_mae: 5.9295
Epoch 2/25
96/96 [==============================] - 1s 15ms/step - loss: 38.7092 - mae: 4.3831 - val_loss: 76.0582 - val_mae: 5.8241
Epoch 3/25
96/96 [==============================] - 1s 15ms/step - loss: 33.6408 - mae: 3.9679 - val_loss: 75.8722 - val_mae: 5.7814
Epoch 4/25
96/96 [==============================] - 1s 15ms/step - loss: 34.9774 - mae: 4.0526 - val_loss: 76.4030 - val_mae: 5.8386
Epoch 5/25
96/96 [==============================] - 1s 14ms/step - loss: 34.9209 - mae: 3.9880 - val_loss: 77.7269 - val_mae: 5.9102
Epoch 6/25
96/96 [==============================] - 1s 16ms/step - loss: 34.9146 - mae: 4.0270 - val_loss: 74.6004 - val_mae: 5.8046
Epoch 7/25
96/96 [==============================] - 1s 15ms/step - loss: 34.5969 - mae: 4.0009 - val_loss: 74.2310 - val_mae: 5.7386
Epoch 8/25
96/96 [==============================] - 1s 15ms/step - l

In [379]:
pd.DataFrame(model.predict(df_merged_test_numpy)).rename( \
                        columns = {0:'AFD',1:'CDU',2:'FDP',3: 'GRUENE',4: 'LINKE', 5:'SPD', 6: 'other'}).round(2)

,AFD,CDU,FDP,GRUENE,LINKE,SPD,other
0,11.61,21.469999,10.83,19.190001,7.61,15.31,10.76
